# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [27]:
# DROP TABLES

songplay_table_drop = "DROP TABLE IF EXISTS fact_songplays ;"
user_table_drop = "DROP TABLE IF EXISTS  dim_users ;"
song_table_drop = "DROP TABLE IF EXISTS  dim_songs ;"
artist_table_drop = "DROP TABLE IF EXISTS  dim_artists ;"
time_table_drop = "DROP TABLE IF EXISTS  dim_time ;"

# CREATE TABLES

songplay_table_create = "CREATE TABLE IF NOT EXISTS fact_songplays ( start_time timestamp, user_id varchar, level varchar, song_id varchar, artist_id varchar, session_id int, location varchar, user_agent varchar);"
user_table_create = "CREATE TABLE IF NOT EXISTS dim_users (user_id varchar, first_name varchar, last_name varchar, gender varchar, level varchar);"
song_table_create = "CREATE TABLE IF NOT EXISTS dim_songs (song_id varchar, title varchar, artist_id varchar, year int, durationInSec int);"
artist_table_create = "CREATE TABLE IF NOT EXISTS dim_artists (artist_id varchar, name varchar, location varchar, latitude varchar, longitude varchar);"
time_table_create = "CREATE TABLE IF NOT EXISTS dim_time (start_time timestamp, hour int, day int, week int, month int, year int, weekday int);"

# INSERT RECORDS

songplay_table_insert = (
    """INSERT INTO fact_songplays ( start_time , user_id , level , song_id , artist_id , session_id , location , user_agent ) VALUES ( %s, %s,%s,%s,%s,%s,%s,%s);
    """
)

user_table_insert = (
    """
    INSERT INTO dim_users (user_id , first_name , last_name , gender , level ) VALUES (%s, %s, %s,%s,%s);
    """
)
song_table_insert = (
    """
    INSERT INTO dim_songs (song_id , title , artist_id , year , durationInSec ) VALUES (%s, %s, %s,%s,%s);
    """
)
artist_table_insert = (
    """
    INSERT INTO dim_artists (artist_id , name , location , latitude , longitude ) VALUES (%s, %s, %s,%s,%s);
    """
)
time_table_insert = (
    """
    INSERT INTO dim_time (start_time , hour , day , week , month , year , weekday ) VALUES (%s, %s, %s,%s,%s,%s,%s);
    """
)

# FIND SONGS

song_select = (
    """
SELECT ds.song_id,ds.artist_id 
FROM dim_songs as ds 
JOIN dim_artists as da 
ON ds.artist_id = da.artist_id
WHERE ds.title = %s and da.name = %s and ds.durationInSec=%s;
    """ 
)

# QUERY LISTS

create_table_queries = [songplay_table_create, user_table_create, song_table_create, artist_table_create, time_table_create]
drop_table_queries = [songplay_table_drop, user_table_drop, song_table_drop, artist_table_drop, time_table_drop]

In [3]:
import psycopg2
from sql_queries import create_table_queries, drop_table_queries


def create_database():
    # connect to default database
    conn = psycopg2.connect("host=127.0.0.1 dbname=studentdb user=student password=student")
    conn.set_session(autocommit=True)
    cur = conn.cursor()
    
    # create sparkify database with UTF8 encoding
    cur.execute("DROP DATABASE IF EXISTS sparkifydb")
    cur.execute("CREATE DATABASE sparkifydb WITH ENCODING 'utf8' TEMPLATE template0")

    # close connection to default database
    conn.close()    
    
    # connect to sparkify database
    conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
    cur = conn.cursor()
    
    return cur, conn


def drop_tables(cur, conn):
    for query in drop_table_queries:
        cur.execute(query)
        conn.commit()


def create_tables(cur, conn):
    for query in create_table_queries:
        cur.execute(query)
        conn.commit()


def main():
    cur, conn = create_database()
    
    drop_tables(cur, conn)
    create_tables(cur, conn)

    conn.close()


if __name__ == "__main__":
    main()

In [4]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [5]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

 

In [6]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [7]:
song_files = get_files("data/song_data/")
#print(song_files)

In [8]:
for filepath in song_files:
    df = pd.read_json(filepath, lines=True)
    #df.head()
    song_data = list([df.values[0][7],df.values[0][8],df.values[0][0],df.values[0][9],df.values[0][5]])
    cur.execute(song_table_insert, song_data)
    conn.commit()

In [9]:
#filepath = song_files[0]

In [10]:
#df = pd.read_json(filepath, lines=True)
#df.head()




## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [11]:
#song_data = list([df.values[0][7],df.values[0][8],df.values[0][0],df.values[0][9],df.values[0][5]])
#song_data

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [12]:
#print (song_table_insert)
#cur.execute(song_table_insert, song_data)
#conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [13]:
for idx,filepath in enumerate(song_files):
    df = pd.read_json(filepath, lines=True)
    #df.head()
    
    artist_data = list([df.values[0][0],df.values[0][4],df.values[0][2],df.values[0][1],df.values[0][3]])
    #print(artist_data)
    cur.execute(artist_table_insert, artist_data)
    conn.commit()


#artist_data = list([df.values[0][0],df.values[0][4],df.values[0][2],df.values[0][1],df.values[0][3]])
#artist_data

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [14]:
#cur.execute(artist_table_insert, artist_data)
#conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [15]:
log_files = get_files("data/log_data/")
#print (log_files)

In [16]:
#filepath = log_files[0]
for idx,filepath in enumerate(log_files):
    df = pd.read_json(filepath, lines=True)
    df = df[df["page"]=="NextSong"]
    
    t = pd.to_datetime(df['ts'], unit='ms')
    time_data = (t,t.dt.hour,t.dt.day,t.dt.week,t.dt.month,t.dt.year,t.dt.weekday)
    column_labels = ("start_time" , "hour" , "day" , "week" , "month" , "year" , "weekday")
    time_df = pd.DataFrame(list(zip(time_data[0],time_data[1],time_data[2],time_data[3],time_data[4],time_data[5],time_data[6])),columns =column_labels)
    for i, row in time_df.iterrows():
        cur.execute(time_table_insert, list(row))
        
    
    user_df = pd.DataFrame(list(zip(df["userId"],df["firstName"],df["lastName"],df["gender"],df["level"])),columns = ("userId","firstName","lastName","gender","level"))
    
    for i, row in user_df.iterrows():
        cur.execute(user_table_insert, row)
    
    conn.commit()
    

In [17]:
#df = pd.read_json(filepath, lines=True)
#df.head()

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [18]:

#df = df[df["page"]=="NextSong"]
#df.head()

In [19]:
#t = pd.to_datetime(df['ts'], unit='ms') 
#t.head()




In [20]:
#time_data = (t,t.dt.hour,t.dt.day,t.dt.week,t.dt.month,t.dt.year,t.dt.weekday)
#column_labels = ("start_time" , "hour" , "day" , "week" , "month" , "year" , "weekday")
#print(column_labels)

In [21]:
#time_df = pd.DataFrame(list(zip(time_data[0],time_data[1],time_data[2],time_data[3],time_data[4],time_data[5],time_data[6])),columns =column_labels)
#time_df.head()

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [22]:
#for i, row in time_df.iterrows():
#    cur.execute(time_table_insert, list(row))
#    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [23]:
#user_df = pd.DataFrame(list(zip(df["userId"],df["firstName"],df["lastName"],df["gender"],df["level"])),columns = ("userId","firstName","lastName","gender","level"))


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [24]:
#for i, row in user_df.iterrows():
#    cur.execute(user_table_insert, row)
#    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [25]:
cur.execute("select * from dim_songs;")
#cur.execute("SELECT ds.song_id,ds.artist_id FROM dim_songs as ds JOIN dim_artists as da ON ds.artist_id = da.artist_id ;")
results = cur.fetchone()
print(results)
# ds.title = %s and da.name = %s and ds.durationInSec=%s

('SOINLJW12A8C13314C', 'City Slickers', 'AR8IEZO1187B99055E', 2008, 150)


In [29]:
for index, row in df.iterrows():
    #print(row.ts)
    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    #print(results)
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None
    
    # insert songplay record
    t = pd.to_datetime(row.ts, unit='ms')
    
    songplay_data = ( t , row.userId , row.level , songid , artistid , row.sessionId , row.location , row.userAgent )
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [60]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.